# imports 

In [10]:
%reset -f

In [19]:
from captum.attr import ShapleyValueSampling
from load_data import load_data
from models.model_wrappers import *
from models.train_models import *
from segmentation import *
from utils import *
import torch
import os
from tqdm import tqdm
import timeit
import sys

# device for torch

In [20]:
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest','QUANT']
}
segmentation_dict = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

# hyper-parameters

In [21]:
# hyperparameters
n_background = 50

# settings
dataset_names = {'gunpoint'}    #{sys.argv[1]}
predictor_names = {'miniRocket'}    #{sys.argv[2]} {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = {"clasp","greedygaussian", "equal", "infogain","nnsegment"} # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} 
background_names =  {"average", "zero","sampling"} #{"average", "zero", "sampling"}
normalization_names = {"default", "normalized"}

demo_mode = True
# demo
if demo_mode:
    dataset_names = {'gunpoint'}
    predictor_names = {"QUANT"}
    segmentation_names = { "equal",'clasp'}
    background_names ={"average","sampling"} #,'sampling'}
    normalization_names = {"default", "normalized"}


# instantiate the dictionary

In [22]:
results = dict.fromkeys(('y_test_true', 'label_mapping', "segments", 'y_test_pred', "attributions"))
for key in results.keys():
    results[key] = dict.fromkeys(dataset_names)
normalization_names = normalization_names | {"default"}


# train model

In [23]:
for dataset_name in dataset_names:
    # init dataset
    # load data
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
    # for debugging only
    if demo_mode:
        X_test = X_test[:2]
        y_test = y_test[:2]

    n_samples, n_chs, ts_length = X_test.shape

    results['y_test_true'][dataset_name] = y_test
    results['label_mapping'][dataset_name] = enc
    results["attributions"][dataset_name] = dict.fromkeys(segmentation_names)
    results["segments"][dataset_name] = dict.fromkeys(segmentation_names)
    results["y_test_pred"][dataset_name] = dict.fromkeys(predictor_names)

    predictor_dict = dict()
    # TODO not to save if in demo mode!
    for predictor_name in predictor_names:
        if demo_mode:
            dataset_name=None

        if predictor_name=='resNet':
            clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
        elif predictor_name=='miniRocket':
            clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="randomForest":
            clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
        elif predictor_name=="QUANT":
            clf, preds = train_QUANT(X_train, y_train, X_test, y_test, dataset_name)
        else:
            raise ValueError("predictor not found")

        predictor_dict[predictor_name] = {"clf": clf, "preds": preds}


training QUANT
accuracy for QUANT is  1.0


In [24]:
def initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results):
	init_segments = np.empty((X_test.shape[0], X_test.shape[1]), dtype=object) if X_test.shape[1] > 1 else (
		np.empty(X_test.shape[0], dtype=object))
	results["segments"][dataset_name][segmentation_name] = init_segments.copy()
	results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
	for predictor_name in predictor_names:
		results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)


def get_sample_info(X_test,y_test,results, id):
	# get current sample and label
	ts, y = X_test[id], torch.tensor(y_test[id:id + 1])
	# get segment and its tensor representation
	current_segments = segmentation_method(ts)[:X_test.shape[1]]
	results['segments'][dataset_name][segmentation_name][i] = current_segments
	mask = get_feature_mask(current_segments, ts.shape[-1])
	mask_list.append(mask)
	ts = torch.tensor(ts).repeat(1, 1, 1)  # TODO use something similar to np.expand_dim?
	ts_list.append(ts)
	y_list.append(y)
	return ts,y,mask


def get_background( background_name, results, normalization_names):

	results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(
		normalization_names)
	# background data
	if background_name == "zero":
		background_dataset = torch.zeros((1,) + X_train.shape[1:])
	elif background_name == "sampling":
		background_dataset = sample_background(X_train, n_background)
	elif background_name == "average":
		background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

	return background_dataset


def get_attribution(ts, mask, background_dataset,y, results ): #    global ts, mask, background_dataset, tmp, y
	# different call depending on predictor type
	if predictor_name in predictors['scikit']:
		# if using random forest flat everything
		if predictor_name == "randomForest":
			ts = ts.reshape(-1, n_chs * ts_length)
			mask = mask.reshape(-1, n_chs * ts_length)
			background_dataset = background_dataset.reshape(-1, n_chs * ts_length)

		tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)

	elif predictor_name in predictors['torch']:
		# if use torch make sure everything is on selected device
		ts = ts.to(device);
		y = y.to(device)
		mask = mask.to(device);
		background_dataset = background_dataset.to(device)
		tmp = SHAP.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset)

	# in case of random forest 'un-flatten' result
	if predictor_name=="randomForest":
		tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

	# lastly store current explanation in the data structure; if sampling store the mean
	results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
		background_name=="sampling" else tmp[0].cpu().numpy()


def get_normalized_results(normalization_names,results):
	if "normalized" in normalization_names:
		weights = np.array(list(map(
			lambda segmentation: list(map(
				lambda channel_segemnts: lengths_to_weights(change_points_to_lengths(channel_segments, X_train.shape[-1])),
				segmentation)),
			results["segments"][dataset_name][segmentation_name])))

		results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["normalized"] = \
			results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"] * weights
	if "default" not in normalization_names:
		del results['attributions'][dataset_name][segmentation_name][predictor_name][background_name]["default"]


In [25]:
starttime = timeit.default_timer()

with torch.no_grad():
    for dataset_name in dataset_names:
        for predictor_name in predictor_names:
            results['y_test_pred'][dataset_name][predictor_name] = predictor_dict[predictor_name]["preds"]
        for segmentation_name in segmentation_names:
            initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results)
            segmentation_method = segmentation_dict[segmentation_name]
            
            ts_list = []
            mask_list = []
            y_list = []
            
            for i in range(n_samples) : 
                ts,y,mask = get_sample_info(X_test,y_test, results, i)
            
            for background_name in background_names:
                background_dataset = get_background( background_name, results, normalization_names)
                for predictor_name in predictor_names:
                    # get clf and initialize attributions
                    clf = predictor_dict[predictor_name]["clf"]
                    init_attributions = np.zeros(X_test.shape, dtype=np.float32)
                    for normalization_name in normalization_names:
                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name] = init_attributions.copy()

                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] else ShapleyValueSampling(forward_classification)
                    
                    with tqdm(total=len(ts_list)) as pbar:
                        for i, (ts, mask, y) in enumerate(zip(ts_list, mask_list, y_list)):
                            # for sampling strategy repeat the ts many times as the background dataset size
                            ts = ts.repeat(background_dataset.shape[0],1,1) if background_name=="sampling" else ts
                            get_attribution (ts, mask, background_dataset,y, results )
                            # update tqdm
                            pbar.update(1)
                            
                    pbar.close()
                    get_normalized_results(normalization_names,results)
                    
print("elapsed time", ( timeit.default_timer() -starttime ) )


100%|██████████| 2/2 [00:11<00:00,  5.83s/it]

elapsed time 35.07191439400049


In [8]:
# dump result to disk
if not demo_mode:
    file_name = "_".join( ("all_results",dataset_name,predictor_name) )
else:
	file_name = "_".join( ("all_results_DEMO_",dataset_name,predictor_name) )
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )

In [26]:
results["y_test_pred"]

{'gunpoint': {'QUANT': array([[0.985, 0.015],
         [0.015, 0.985]])}}

In [11]:
import scipy
scipy.stats.entropy([0.5,], base=None)

0.0